# Arithmetic Expressions

This tutorial demonstrates automatic arithmetic operation management by the synthesis engine. It synthesizes a complex arithmetic expression, where uncomputation procedure, together with initialization and reuse of auxiliary qubits, are all automated. Given different **global** width or depth constraints results in different circuits.

Define a quantum model that applies some quantum arithmetic operation on `QNum` variables.

In [1]:
from classiq import Output, QBit, QNum, allocate, create_model, qfunc, synthesize
from classiq.qmod.symbolic import max


@qfunc
def main(z: Output[QNum]):
    x = QNum("x")
    y = QNum("y")
    allocate(2, x)
    allocate(1, y)
    z |= (2 * x + y + max(3 * y, 2)) > 4


qmod = create_model(main)

You can try different optimization scenarios, below we introduce two examples:
1. Optimizing over depth and constraining the maximal width to 9 qubits.
2. Optimizing over depth and constraining the maximal width to 12 qubits.

In [2]:
from classiq import set_constraints, set_preferences, show, write_qmod
from classiq.model import Constraints, Preferences

constraints = Constraints(optimization_parameter="depth", max_width=9)

preferences = Preferences(random_seed=424788457)
qmod = set_constraints(qmod, constraints)
qmod = set_preferences(qmod, preferences)

qprog = synthesize(qmod)
show(qprog)

write_qmod(qmod, "arithmetic_demo_9_qubits")

Opening: https://platform.classiq.io/circuit/5c9aeae6-4771-468d-8291-2c095acbbc67?version=0.38.0.dev42%2Bfd36e2c41c


Change the quantum model constraint to treat the second scenario:

In [3]:
constraints = Constraints(optimization_parameter="depth", max_width=12)
qmod = set_constraints(qmod, constraints)

qprog = synthesize(qmod)
show(qprog)

write_qmod(qmod, "arithmetic_demo_12_qubits")

Opening: https://platform.classiq.io/circuit/81b43d3e-bfad-4ace-8512-198695f706d6?version=0.38.0.dev42%2Bfd36e2c41c
